
#### Intro
This example uses Approximate Nearest Neighbor Search (ANN) using [Hnswlib](https://github.com/nmslib/hnswlib/).

Install it with: `pip install hnswlib`

The embeddings model is 'msmarco-distilbert-base-v4' from SBERT

The dataset we use is our own, (at ../dish-washer-data.csv); that gives us a way to compare with naive search



In [1]:
import os
import csv
import pickle
import time
import hnswlib
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer, util
import numpy as np

/home/auro/anaconda3/envs/openai/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = 'msmarco-distilbert-base-v4'
embeddings_model = SentenceTransformer(model_name)
embedding_size = 768    #Size of embeddings
top_k_hits = 5         #Output k hits

In [3]:
def get_embedding(text: str, model: SentenceTransformer) -> list[float]:
    embeddings = model.encode([text])
    return embeddings

def get_doc_embedding(text: str, model: SentenceTransformer) -> list[float]:
    return get_embedding(text, model)

def get_query_embedding(text: str, model: SentenceTransformer) -> list[float]:
    return get_embedding(text, model)

def compute_doc_embeddings(df: pd.DataFrame, model: SentenceTransformer) -> dict[tuple[str, str], list[float]]:
    """
    Create an embedding for each row in the dataframe.
    Return a dictionary that maps between each embedding vector and the index of the row that it corresponds to.
    """
    return {
        idx: get_doc_embedding(r.content.replace("\n", " "), model) for idx, r in df.iterrows()
    }

In [4]:
embeddings_cache_path = f"dishwasher-repair-manual-embeddings-{model_name.replace('/', '_')}.pkl"

#Check if embeddings cache path exists
if not os.path.exists(embeddings_cache_path):
    df = pd.read_csv('../dish-washer-data.csv')
    df["tokens"] = pd.to_numeric(df["tokens"])  # convert column "tokens" of a DataFrame
    df = df.set_index(["title", "heading"])
    print(f"{len(df)} rows in the data.")
    print(df.sample(10))
    # TODO get some stats on max/min content length
    corpus_size = df.shape[0]

    # This could take a bit of time
    print("Encoding the corpus. This might take a while...")
    corpus_embeddings = compute_doc_embeddings(df, embeddings_model)

    print("Store file...")
    with open(embeddings_cache_path, "wb") as fOut:
        pickle.dump({'sentences': df, 'embeddings': corpus_embeddings}, fOut)
else:
    print("Loading pre-computed embeddings from disc...")
    with open(embeddings_cache_path, "rb") as f_in:
        cache_data = pickle.load(f_in)
        corpus_sentences = cache_data['sentences']
        corpus_embeddings = cache_data['embeddings']
        corpus_embeddings = [v for k, v in corpus_embeddings.items()]
        print("Loading done.")
        print(f'Length, corpus sentences:{len(corpus_sentences)}')
        print(f'Length, corpus embeddings:{len(corpus_embeddings)}')
        print(f'one vector dimension: {len(corpus_embeddings[0][0])}')

        print(f"Sentence samples\n {corpus_sentences.sample(5)}")
        


Loading pre-computed embeddings from disc...
Loading done.
Length, corpus sentences:149
Length, corpus embeddings:149
one vector dimension: 768
Sentence samples
                                                              content  tokens
title     heading                                                           
Chapter_5 265       A macerator is located outside the pump scree...     162
Chapter_3 160       Fortunately, they are pretty easy to open and...      99
Chapter_2 129       Don't let the word "continuity" scare you. It...     112
Chapter_6 388       Testing switch blocks is much like testing ti...     102
Chapter_5 273       Feel the action of the flapper valve. If it i...      96


In [5]:
#Defining our hnswlib index
index_path = "./hnswlib.index"
index = hnswlib.Index(space = 'cosine', dim = embedding_size)

if os.path.exists(index_path):
    print("Loading index...")
    index.load_index(index_path)
    print('Loading index done.')
else:
    ### Create the HNSWLIB index
    print("Start creating HNSWLIB index")
    # TODO check is ef_consturction and M and appropriate for this dataset
    index.init_index(max_elements = len(corpus_embeddings), ef_construction = 400, M = 64)

    print(list(range(len(corpus_embeddings))))
    # Then we train the index to find a suitable clustering
    corpus_embeddings = np.array(corpus_embeddings).squeeze()
    print('shape', corpus_embeddings.shape)
    # index.add_items(corpus_embeddings, list(range(len(corpus_embeddings))))
    index.add_items(corpus_embeddings, list(corpus_sentences.index))

    print("Saving index to:", index_path)
    index.save_index(index_path)

# Controlling the recall by setting ef:
index.set_ef(50)  # ef should always be > top_k_hits



Loading index...
Loading index done.


In [21]:
# Choose questions
# inp_question = "Why is my dishwasher leaking?"
inp_question = "Why do we use a dishwasher?"

In [22]:
start_time = time.time()
question_embedding = embeddings_model.encode(inp_question)

# Use hnswlib knn_query method to find the top_k_hits
corpus_ids, distances = index.knn_query(question_embedding, k=top_k_hits)

# Extract corpus ids and scores for the query
hits = [{'corpus_id': id, 'score': 1 - score} for id, score in zip(corpus_ids[0], distances[0])]
hits = sorted(hits, key=lambda x: x['score'], reverse=True)
end_time = time.time()
print(f'Hits\n, {hits}')

print("Input question:", inp_question)
print("Results (after {:.3f} seconds):".format(end_time-start_time))
for hit in hits[0:top_k_hits]:
    print(f"\t{hit['score']:.3f}\t{corpus_sentences.iloc[hit['corpus_id']].content}")
    



Hits
, [{'corpus_id': 1, 'score': 0.7641448974609375}, {'corpus_id': 106, 'score': 0.6384871602058411}, {'corpus_id': 0, 'score': 0.6073850393295288}, {'corpus_id': 15, 'score': 0.6050634980201721}, {'corpus_id': 3, 'score': 0.5990546941757202}]
Input question: Why do we use a dishwasher?
Results (after 0.018 seconds):
	0.764	 The main reason dishwashers exist is that they allow dishes to be washed in water much hotter than you can use when washing dishes by hand. This allows greater grease-cutting and sterilization of the dishes. They are NOT made to operate under cold water conditions or to ingest your disgusting, moldy leftovers, no matter what the sales literature says. And using cheap soap and hard water (without making some adjustments) can shorten their lives considerably.
	0.638	 Nowadays, dishwashers are being made as efficient as possible, due in no small part to government energy efficiency requirements. Heating water can use a lot of energy, so designers are mimimizing wate

In [23]:
# Approximate Nearest Neighbor (ANN) is not exact, it might miss entries with high cosine similarity
# Here, we compute the recall of ANN compared to the exact results
correct_hits = util.semantic_search(torch.tensor(question_embedding), 
                                    torch.tensor(corpus_embeddings).squeeze(), 
                                    top_k=top_k_hits)[0]

print(f'Correct hits:\n {correct_hits}')

for hit in correct_hits[0:top_k_hits]:
    print(f"\t{hit['score']:.3f}\t{corpus_sentences.iloc[hit['corpus_id']].content}")

correct_hits_ids = set([hit['corpus_id'] for hit in correct_hits])

ann_corpus_ids = set([hit['corpus_id'] for hit in hits])
if len(ann_corpus_ids) != len(correct_hits_ids):
    print("Approximate Nearest Neighbor returned a different number of results than expected")

recall = len(ann_corpus_ids.intersection(correct_hits_ids)) / len(correct_hits_ids)
print("\nApproximate Nearest Neighbor Recall@{}: {:.2f}".format(top_k_hits, recall * 100))

if recall < 1:
    print("Missing results:")
    for hit in correct_hits[0:top_k_hits]:
        if hit['corpus_id'] not in ann_corpus_ids:
            print("\t{:.3f}\t{}".format(hit['score'], corpus_sentences[hit['corpus_id']]))
print("\n\n========\n")


Correct hits:
 [{'corpus_id': 1, 'score': 0.7641448974609375}, {'corpus_id': 106, 'score': 0.6384871006011963}, {'corpus_id': 0, 'score': 0.6073852181434631}, {'corpus_id': 15, 'score': 0.6050636172294617}, {'corpus_id': 3, 'score': 0.5990549325942993}]
	0.764	 The main reason dishwashers exist is that they allow dishes to be washed in water much hotter than you can use when washing dishes by hand. This allows greater grease-cutting and sterilization of the dishes. They are NOT made to operate under cold water conditions or to ingest your disgusting, moldy leftovers, no matter what the sales literature says. And using cheap soap and hard water (without making some adjustments) can shorten their lives considerably.
	0.638	 Nowadays, dishwashers are being made as efficient as possible, due in no small part to government energy efficiency requirements. Heating water can use a lot of energy, so designers are mimimizing water usage and heater operation. The trick is in achieving a balance; 